In [1]:
import os
import sys

sys.path.append('..')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
def read_baard_results(data, model, attacks):
    list_df = []
    for i in range(5):
        for a in attacks:
            path = os.path.join(
                '..','results', 'result_{}'.format(i), '{}_{}'.format(data, model), 'results', 
                '{}_{}_{}_baard.csv'.format(data, model, a))
            df = pd.read_csv(path)
            df = df.drop('Unnamed: 0', axis=1)
            list_df.append(df)
    df = pd.concat(list_df, axis=0, ignore_index=True)
    df = df.groupby(['data', 'model', 'attack', 'adv_param'], as_index=False).agg({
        'acc_no_def': ['mean', 'std'],
        'acc_on_adv_1': ['mean', 'std'],
        'fpr_1': ['mean', 'std'],
        'acc_on_adv_2': ['mean', 'std'],
        'fpr_2': ['mean', 'std'],
        'acc_on_adv_3': ['mean', 'std'],
        'fpr_3': ['mean', 'std']})
    df.columns = ["_".join(x) if x[1] != '' else x[0] for x in df.columns]
    return df

In [5]:
def reduce_baard(df):
    df = df.rename(columns={
        'acc_on_adv_3_mean': 'acc_on_adv_mean',
        'acc_on_adv_3_std': 'acc_on_adv_std',
        'fpr_3_mean': 'fpr_mean',
        'fpr_3_std': 'fpr_std',
    })
    df['defence'] = 'baard'
    return df[['data', 'model', 'attack', 'adv_param', 'defence', 'acc_no_def_mean', 'acc_no_def_std', 
              'acc_on_adv_mean', 'acc_on_adv_std', 'fpr_mean', 'fpr_std']]

In [6]:
def merge_results(data, model, attacks, defences):
    list_df = []
    for i in range(5):
        for a in attacks:
            for d in defences:
                p_mnist_dnn_apgd2_baard = os.path.join(
                    '..', 'results', 'result_{}'.format(i), '{}_{}'.format(data, model), 'results', 
                    '{}_{}_{}_{}.csv'.format(data, model, a, d))
                df = pd.read_csv(p_mnist_dnn_apgd2_baard)
                df = df.drop('Unnamed: 0', axis=1)
                df['defence'] = d
                list_df.append(df)
    df = pd.concat(list_df, axis=0, ignore_index=True)
    df = df.groupby(['data', 'model', 'attack', 'adv_param', 'defence'], as_index=False).agg({
        'acc_no_def': ['mean', 'std'],
        'acc_on_adv': ['mean', 'std'],
        'fpr': ['mean', 'std']
    })
    df.columns = ["_".join(x) if x[1] != '' else x[0] for x in df.columns]
    return df

## MNIST DNN

In [7]:
attacks = ['fgsm', 'apgd', 'apgd2', 'cw2', 'deepfool', 'line']

mnist_baard = read_baard_results('mnist', 'dnn', attacks)
mnist_baard.head()

,data,model,attack,adv_param,acc_no_def_mean,acc_no_def_std,acc_on_adv_1_mean,acc_on_adv_1_std,fpr_1_mean,fpr_1_std,acc_on_adv_2_mean,acc_on_adv_2_std,fpr_2_mean,fpr_2_std,acc_on_adv_3_mean,acc_on_adv_3_std,fpr_3_mean,fpr_3_std
0,mnist,dnn,apgd,0.063,0.8618,0.004817,1.0,0.0,0.006,0.002345,1.0,0.0,0.0176,0.002881,1.0,0.0,0.0272,0.003271
1,mnist,dnn,apgd,0.100,0.5222,0.022477,1.0,0.0,0.006,0.002345,1.0,0.0,0.0176,0.002881,1.0,0.0,0.0272,0.003271
2,mnist,dnn,apgd,0.300,0.0000,0.000000,1.0,0.0,0.006,0.002345,1.0,0.0,0.0176,0.002881,1.0,0.0,0.0272,0.003271
3,mnist,dnn,apgd,0.600,0.0000,0.000000,1.0,0.0,0.006,0.002345,1.0,0.0,0.0176,0.002881,1.0,0.0,0.0272,0.003271
4,mnist,dnn,apgd,1.000,0.0000,0.000000,1.0,0.0,0.006,0.002345,1.0,0.0,0.0176,0.002881,1.0,0.0,0.0272,0.003271


In [8]:
mnist_baard_reduce = reduce_baard(mnist_baard)
mnist_baard_reduce.head()

,data,model,attack,adv_param,defence,acc_no_def_mean,acc_no_def_std,acc_on_adv_mean,acc_on_adv_std,fpr_mean,fpr_std
0,mnist,dnn,apgd,0.063,baard,0.8618,0.004817,1.0,0.0,0.0272,0.003271
1,mnist,dnn,apgd,0.100,baard,0.5222,0.022477,1.0,0.0,0.0272,0.003271
2,mnist,dnn,apgd,0.300,baard,0.0000,0.000000,1.0,0.0,0.0272,0.003271
3,mnist,dnn,apgd,0.600,baard,0.0000,0.000000,1.0,0.0,0.0272,0.003271
4,mnist,dnn,apgd,1.000,baard,0.0000,0.000000,1.0,0.0,0.0272,0.003271


In [9]:
attacks = ['fgsm', 'apgd', 'apgd2', 'cw2', 'deepfool', 'line']
defences = ['fs', 'lid', 'magnet', 'rc']

mnist = merge_results('mnist', 'dnn', attacks, defences)
mnist.head()

,data,model,attack,adv_param,defence,acc_no_def_mean,acc_no_def_std,acc_on_adv_mean,acc_on_adv_std,fpr_mean,fpr_std
0,mnist,dnn,apgd,0.063,fs,0.8618,0.004817,0.9138,0.010986,0.0528,0.012558
1,mnist,dnn,apgd,0.063,lid,0.8618,0.004817,0.8618,0.004817,0.0000,0.000000
2,mnist,dnn,apgd,0.063,magnet,0.8618,0.004817,1.0000,0.000000,0.0028,0.002168
3,mnist,dnn,apgd,0.063,rc,0.8618,0.004817,0.8952,0.009985,0.0008,0.000837
4,mnist,dnn,apgd,0.100,fs,0.5222,0.022477,0.9368,0.016619,0.0528,0.012558


In [10]:
mnist = pd.concat([mnist_baard_reduce, mnist], axis=0, ignore_index=True)
# mnist = mnist.sort_values(by=['data', 'model', 'attack', 'adv_param', 'acc_on_adv_mean', 'fpr_mean'], 
#                ascending=[True, True, True, True, False, True])
mnist = mnist.sort_values(by=['data', 'model', 'attack', 'adv_param'], 
               ascending=[True, True, True, True])
mnist.head()

,data,model,attack,adv_param,defence,acc_no_def_mean,acc_no_def_std,acc_on_adv_mean,acc_on_adv_std,fpr_mean,fpr_std
0,mnist,dnn,apgd,0.063,baard,0.8618,0.004817,1.0000,0.000000,0.0272,0.003271
25,mnist,dnn,apgd,0.063,fs,0.8618,0.004817,0.9138,0.010986,0.0528,0.012558
26,mnist,dnn,apgd,0.063,lid,0.8618,0.004817,0.8618,0.004817,0.0000,0.000000
27,mnist,dnn,apgd,0.063,magnet,0.8618,0.004817,1.0000,0.000000,0.0028,0.002168
28,mnist,dnn,apgd,0.063,rc,0.8618,0.004817,0.8952,0.009985,0.0008,0.000837


In [11]:
mnist_baard.to_csv('mnist_dnn_baard.csv', index=False)
mnist.to_csv('mnist_dnn.csv', index=False)

## CIFAR10 ResNet

In [12]:
attacks = ['fgsm', 'apgd', 'apgd2', 'cw2', 'deepfool', 'line']
cifar10_resnet_baard = read_baard_results('cifar10', 'resnet', attacks)
cifar10_resnet_baard.head()

,data,model,attack,adv_param,acc_no_def_mean,acc_no_def_std,acc_on_adv_1_mean,acc_on_adv_1_std,fpr_1_mean,fpr_1_std,acc_on_adv_2_mean,acc_on_adv_2_std,fpr_2_mean,fpr_2_std,acc_on_adv_3_mean,acc_on_adv_3_std,fpr_3_mean,fpr_3_std
0,cifar10,resnet,apgd,0.031,0.0004,0.000548,0.0322,0.008379,0.0232,0.004382,0.0704,0.014943,0.043,0.011467,0.1554,0.032929,0.0588,0.014342
1,cifar10,resnet,apgd,0.100,0.0000,0.000000,0.0410,0.021459,0.0232,0.004382,0.0780,0.026077,0.043,0.011467,0.1530,0.046109,0.0588,0.014342
2,cifar10,resnet,apgd,0.300,0.0000,0.000000,0.1614,0.099483,0.0232,0.004382,0.7842,0.058845,0.043,0.011467,0.7950,0.060452,0.0588,0.014342
3,cifar10,resnet,apgd,0.600,0.0000,0.000000,0.7612,0.173413,0.0232,0.004382,0.9994,0.000894,0.043,0.011467,0.9994,0.000894,0.0588,0.014342
4,cifar10,resnet,apgd,1.000,0.0000,0.000000,0.8986,0.121587,0.0232,0.004382,1.0000,0.000000,0.043,0.011467,1.0000,0.000000,0.0588,0.014342


In [13]:
cifar10_resnet_reduce = reduce_baard(cifar10_resnet_baard)
cifar10_resnet_reduce.head()

,data,model,attack,adv_param,defence,acc_no_def_mean,acc_no_def_std,acc_on_adv_mean,acc_on_adv_std,fpr_mean,fpr_std
0,cifar10,resnet,apgd,0.031,baard,0.0004,0.000548,0.1554,0.032929,0.0588,0.014342
1,cifar10,resnet,apgd,0.100,baard,0.0000,0.000000,0.1530,0.046109,0.0588,0.014342
2,cifar10,resnet,apgd,0.300,baard,0.0000,0.000000,0.7950,0.060452,0.0588,0.014342
3,cifar10,resnet,apgd,0.600,baard,0.0000,0.000000,0.9994,0.000894,0.0588,0.014342
4,cifar10,resnet,apgd,1.000,baard,0.0000,0.000000,1.0000,0.000000,0.0588,0.014342


In [14]:
attacks = ['fgsm', 'apgd', 'apgd2', 'cw2', 'deepfool', 'line']
defences = ['fs', 'lid', 'magnet', 'rc']

cifar10_resnet = merge_results('cifar10', 'resnet', attacks, defences)
cifar10_resnet.head()

,data,model,attack,adv_param,defence,acc_no_def_mean,acc_no_def_std,acc_on_adv_mean,acc_on_adv_std,fpr_mean,fpr_std
0,cifar10,resnet,apgd,0.031,fs,0.0004,0.000548,0.8940,0.123221,0.0602,0.016962
1,cifar10,resnet,apgd,0.031,lid,0.0004,0.000548,0.0310,0.068202,0.0118,0.026386
2,cifar10,resnet,apgd,0.031,magnet,0.0004,0.000548,0.2158,0.148549,0.0020,0.000000
3,cifar10,resnet,apgd,0.031,rc,0.0004,0.000548,0.0012,0.001304,0.0030,0.001225
4,cifar10,resnet,apgd,0.100,fs,0.0000,0.000000,0.9692,0.048365,0.0602,0.016962


In [15]:
cifar10_resnet = pd.concat([cifar10_resnet_reduce, cifar10_resnet], axis=0, ignore_index=True)
# cifar10_resnet = cifar10_resnet.sort_values(by=['data', 'model', 'attack', 'adv_param', 'acc_on_adv_mean', 'fpr_mean'], 
#                ascending=[True, True, True, True, False, True])
cifar10_resnet = cifar10_resnet.sort_values(by=['data', 'model', 'attack', 'adv_param'], 
               ascending=[True, True, True, True])
cifar10_resnet.head()

,data,model,attack,adv_param,defence,acc_no_def_mean,acc_no_def_std,acc_on_adv_mean,acc_on_adv_std,fpr_mean,fpr_std
0,cifar10,resnet,apgd,0.031,baard,0.0004,0.000548,0.1554,0.032929,0.0588,0.014342
25,cifar10,resnet,apgd,0.031,fs,0.0004,0.000548,0.8940,0.123221,0.0602,0.016962
26,cifar10,resnet,apgd,0.031,lid,0.0004,0.000548,0.0310,0.068202,0.0118,0.026386
27,cifar10,resnet,apgd,0.031,magnet,0.0004,0.000548,0.2158,0.148549,0.0020,0.000000
28,cifar10,resnet,apgd,0.031,rc,0.0004,0.000548,0.0012,0.001304,0.0030,0.001225


In [16]:
cifar10_resnet_baard.to_csv('cifar10_resnet_baard.csv', index=False)
cifar10_resnet.to_csv('cifar10_resnet.csv', index=False)

## CIFAR10 VGG

In [17]:
attacks = ['fgsm', 'apgd', 'apgd2', 'cw2', 'deepfool', 'line']
cifar10_vgg_baard = read_baard_results('cifar10', 'vgg', attacks)
cifar10_vgg_reduce = reduce_baard(cifar10_vgg_baard)

defences = ['fs', 'lid', 'magnet', 'rc']
cifar10_vgg = merge_results('cifar10', 'vgg', attacks, defences)

cifar10_vgg = pd.concat([cifar10_vgg_reduce, cifar10_vgg], axis=0, ignore_index=True)
# cifar10_vgg = cifar10_resnet.sort_values(by=['data', 'model', 'attack', 'adv_param', 'acc_on_adv_mean', 'fpr_mean'], 
#                ascending=[True, True, True, True, False, True])
cifar10_vgg = cifar10_resnet.sort_values(by=['data', 'model', 'attack', 'adv_param'], 
               ascending=[True, True, True, True])
cifar10_vgg.head()

,data,model,attack,adv_param,defence,acc_no_def_mean,acc_no_def_std,acc_on_adv_mean,acc_on_adv_std,fpr_mean,fpr_std
0,cifar10,resnet,apgd,0.031,baard,0.0004,0.000548,0.1554,0.032929,0.0588,0.014342
25,cifar10,resnet,apgd,0.031,fs,0.0004,0.000548,0.8940,0.123221,0.0602,0.016962
26,cifar10,resnet,apgd,0.031,lid,0.0004,0.000548,0.0310,0.068202,0.0118,0.026386
27,cifar10,resnet,apgd,0.031,magnet,0.0004,0.000548,0.2158,0.148549,0.0020,0.000000
28,cifar10,resnet,apgd,0.031,rc,0.0004,0.000548,0.0012,0.001304,0.0030,0.001225


In [18]:
cifar10_vgg_baard.to_csv('cifar10_vgg_baard.csv', index=False)
cifar10_vgg.to_csv('cifar10_vgg.csv', index=False)

## DNN on numeric datasets

### Banknote

In [19]:
attacks = ['fgsm', 'apgd', 'apgd2', 'cw2']
defences = ['lid', 'magnet', 'rc']

In [20]:
banknote_dnn_baard = read_baard_results('banknote', 'dnn', attacks)
banknote_dnn_reduce = reduce_baard(banknote_dnn_baard)

banknote_dnn = merge_results('banknote', 'dnn', attacks, defences)
banknote_dnn = pd.concat([banknote_dnn_reduce, banknote_dnn], axis=0, ignore_index=True)
banknote_dnn = banknote_dnn.sort_values(by=['data', 'model', 'attack', 'adv_param'], ascending=True)
banknote_dnn.head()

,data,model,attack,adv_param,defence,acc_no_def_mean,acc_no_def_std,acc_on_adv_mean,acc_on_adv_std,fpr_mean,fpr_std
0,banknote,dnn,apgd,0.05,baard,0.732430,0.030555,0.923216,0.012671,0.009091,0.007457
23,banknote,dnn,apgd,0.05,lid,0.732430,0.030555,0.834396,0.031978,0.047516,0.019185
24,banknote,dnn,apgd,0.05,magnet,0.732430,0.030555,0.658426,0.052355,0.003030,0.006776
25,banknote,dnn,apgd,0.05,rc,0.732430,0.030555,0.735445,0.028710,0.005066,0.005076
1,banknote,dnn,apgd,0.10,baard,0.184929,0.025798,0.660400,0.051242,0.009091,0.007457


In [21]:
banknote_dnn_baard.to_csv('banknote_dnn_baard.csv', index=False)
banknote_dnn.to_csv('banknote_dnn.csv', index=False)

### Breastcancer

In [22]:
breastcancer_dnn_baard = read_baard_results('breastcancer', 'dnn', attacks)
breastcancer_dnn_reduce = reduce_baard(breastcancer_dnn_baard)

breastcancer_dnn = merge_results('breastcancer', 'dnn', attacks, defences)
breastcancer_dnn = pd.concat([breastcancer_dnn_reduce, breastcancer_dnn], axis=0, ignore_index=True)
breastcancer_dnn = breastcancer_dnn.sort_values(by=['data', 'model', 'attack', 'adv_param'], ascending=True)

breastcancer_dnn_baard.to_csv('breastcancer_dnn_baard.csv', index=False)
breastcancer_dnn.to_csv('breastcancer_dnn.csv', index=False)

### HTRU2

In [23]:
htru2_dnn_baard = read_baard_results('htru2', 'dnn', attacks)
htru2_dnn_reduce = reduce_baard(htru2_dnn_baard)

htru2_dnn = merge_results('htru2', 'dnn', attacks, defences)
htru2_dnn = pd.concat([htru2_dnn_reduce, htru2_dnn], axis=0, ignore_index=True)
htru2_dnn = htru2_dnn.sort_values(by=['data', 'model', 'attack', 'adv_param'], ascending=True)

htru2_dnn_baard.to_csv('htru2_dnn_baard.csv', index=False)
htru2_dnn.to_csv('htru2_dnn.csv', index=False)

## SVM

### Banknote

In [24]:
attacks = ['fgsm', 'bim', 'boundary']
defences = ['magnet', 'rc']

In [25]:
banknote_svm_baard = read_baard_results('banknote', 'svm', attacks)
banknote_svm_reduce = reduce_baard(banknote_svm_baard)

banknote_svm = merge_results('banknote', 'svm', attacks, defences)
banknote_svm = pd.concat([banknote_svm_reduce, banknote_svm], axis=0, ignore_index=True)
banknote_svm = banknote_svm.sort_values(by=['data', 'model', 'attack', 'adv_param'], ascending=True)

banknote_svm_baard.to_csv('banknote_svm_baard.csv', index=False)
banknote_svm.to_csv('banknote_svm.csv', index=False)

### Breastcancer

In [26]:
breastcancer_svm_baard = read_baard_results('breastcancer', 'svm', attacks)
breastcancer_svm_reduce = reduce_baard(breastcancer_svm_baard)

breastcancer_svm = merge_results('breastcancer', 'svm', attacks, defences)
breastcancer_svm = pd.concat([breastcancer_svm_reduce, breastcancer_svm], axis=0, ignore_index=True)
breastcancer_svm = breastcancer_svm.sort_values(by=['data', 'model', 'attack', 'adv_param'], ascending=True)

breastcancer_svm_baard.to_csv('breastcancer_svm_baard.csv', index=False)
breastcancer_svm.to_csv('breastcancer_svm.csv', index=False)

### HTRU2

In [27]:
htru2_svm_baard = read_baard_results('htru2', 'svm', attacks)
htru2_svm_reduce = reduce_baard(htru2_svm_baard)

htru2_svm = merge_results('htru2', 'svm', attacks, defences)
htru2_svm = pd.concat([htru2_svm_reduce, htru2_svm], axis=0, ignore_index=True)
htru2_svm = htru2_svm.sort_values(by=['data', 'model', 'attack', 'adv_param'], ascending=True)

htru2_svm_baard.to_csv('htru2_svm_baard.csv', index=False)
htru2_svm.to_csv('htru2_svm.csv', index=False)

## Tree

### Banknote

In [28]:
attacks = ['tree', 'boundary']
defences = ['magnet', 'rc']

In [29]:
banknote_tree_baard = read_baard_results('banknote', 'tree', attacks)
banknote_tree_reduce = reduce_baard(banknote_tree_baard)

banknote_tree = merge_results('banknote', 'tree', attacks, defences)
banknote_tree = pd.concat([banknote_tree_reduce, banknote_tree], axis=0, ignore_index=True)
banknote_tree = banknote_tree.sort_values(by=['data', 'model', 'attack', 'adv_param'], ascending=True)

banknote_tree_baard.to_csv('banknote_tree_baard.csv', index=False)
banknote_tree.to_csv('banknote_tree.csv', index=False)

### Breastcancer

In [30]:
breastcancer_tree_baard = read_baard_results('breastcancer', 'tree', attacks)
breastcancer_tree_reduce = reduce_baard(breastcancer_tree_baard)

breastcancer_tree = merge_results('breastcancer', 'tree', attacks, defences)
breastcancer_tree = pd.concat([breastcancer_tree_reduce, breastcancer_tree], axis=0, ignore_index=True)
breastcancer_tree = breastcancer_tree.sort_values(by=['data', 'model', 'attack', 'adv_param'], ascending=True)

breastcancer_tree_baard.to_csv('breastcancer_tree_baard.csv', index=False)
breastcancer_tree.to_csv('breastcancer_tree.csv', index=False)

### HTRU2

In [31]:
htru2_tree_baard = read_baard_results('htru2', 'tree', attacks)
htru2_tree_reduce = reduce_baard(htru2_tree_baard)

htru2_tree = merge_results('htru2', 'tree', attacks, defences)
htru2_tree = pd.concat([htru2_tree_reduce, htru2_tree], axis=0, ignore_index=True)
htru2_tree = htru2_tree.sort_values(by=['data', 'model', 'attack', 'adv_param'], ascending=True)

htru2_tree_baard.to_csv('htru2_tree_baard.csv', index=False)
htru2_tree.to_csv('htru2_tree.csv', index=False)